## Setup

In [18]:
from dotenv import load_dotenv
from utils import chat_interface
from agentic.tools import db_tools

In [32]:
import asyncio
import sys
import os
from pathlib import Path

from langchain_core.messages import HumanMessage
from langchain_core.tools import StructuredTool

import nest_asyncio
nest_asyncio.apply()

# Add project paths
sys.path.append(str(Path.cwd()))
sys.path.append(str(Path.cwd() / "agentic"))

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient

# Testing MCP Server

In [28]:
# Test 1: Basic MCP Connection and Tool Discovery
print("\n📡 TEST 1: MCP Server Connection & Tool Discovery")
print("-" * 50)

# Server configuration


async def test_mcp_connection():
    """Test basic MCP connection and list all available tools"""
    print("🔌 Connecting to MCP Server...")

    errlog = sys.stderr
    try:
        sys.stderr.fileno()
    except (io.UnsupportedOperation, AttributeError):
        errlog = open(os.devnull, 'w')

    server_params = StdioServerParameters(
    command="python",
    args=["mcp_server.py"],
    env=None
    )

    async with stdio_client(server_params, errlog = errlog) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()
            print("✅ MCP Server connection established")

            # List available tools
            tools_result = await session.list_tools()
            print(f"\n🛠️  Available Tools ({len(tools_result.tools)}):")

            tools_info = {}
            for i, tool in enumerate(tools_result.tools, 1):
                print(f"   {i}. {tool.name}")
                print(f"      Description: {tool.description}")
                print(f"      Parameters: {list(tool.inputSchema.get('properties', {}).keys())}")
                print()

                # Store tool info for later use
                tools_info[tool.name] = {
                    'description': tool.description,
                    'parameters': tool.inputSchema.get('properties', {}),
                    'required': tool.inputSchema.get('required', [])
                }

            return tools_info



📡 TEST 1: MCP Server Connection & Tool Discovery
--------------------------------------------------


In [29]:
nest_asyncio.apply()

tools_info = await test_mcp_connection()

🔌 Connecting to MCP Server...
✅ MCP Server connection established

🛠️  Available Tools (7):
   1. lookup_customer
      Description: Retrieves customer profile by email (ID, Name, Block Status).
      Parameters: ['email']

   2. get_user_subscription
      Description: Fetches subscription details (Tier, Status, Quota) for a user ID.
      Parameters: ['user_id']

   3. cancel_subscription_action
      Description: Cancels a user's subscription immediately.
      Parameters: ['user_id']

   4. get_available_experiences
      Description: Lists upcoming experiences that have available slots.
      Parameters: []

   5. get_user_reservations
      Description: Lists existing reservations for a user.
      Parameters: ['user_id']

   6. create_reservation_action
      Description: Books an experience for a user if slots are available.
      Parameters: ['user_id', 'experience_id']

   7. search_knowledge_base
      Description: Semantic search for knowledge base articles.
      Parameter

In [30]:
print("\n🔧 TEST 2: Individual Tool Testing")
print("-" * 50)

async def test_individual_tools():
    """Test each MCP tool individually with sample data"""

    # Required stderr configuration for Jupyter
    errlog = sys.stderr
    try:
        sys.stderr.fileno()
    except (io.UnsupportedOperation, AttributeError):
        errlog = open(os.devnull, 'w')

    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"],
        env=None
    )

    async with stdio_client(server_params, errlog=errlog) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            test_cases = [
                {
                    "tool": "lookup_customer",
                    "args": {"email": "john.doe@example.com"},
                    "description": "Looking up customer by email"
                },
                {
                    "tool": "get_available_experiences",
                    "args": {},
                    "description": "Getting available experiences"
                },
                {
                    "tool": "search_knowledge_base",
                    "args": {"query": "subscription policy"},
                    "description": "Searching knowledge base"
                }
            ]

            results = {}

            for test_case in test_cases:
                tool_name = test_case["tool"]
                args = test_case["args"]
                desc = test_case["description"]

                print(f"🔍 Testing: {tool_name}")
                print(f"   {desc}")
                print(f"   Args: {args}")

                try:
                    result = await session.call_tool(tool_name, arguments=args)
                    content = "\n".join([c.text for c in result.content if c.type == "text"])
                    results[tool_name] = content
                    print(f"   ✅ Result: {content[:100]}...")
                    print()
                except Exception as e:
                    print(f"   ❌ Error: {str(e)}")
                    results[tool_name] = f"Error: {str(e)}"
                    print()

            return results

# Run individual tool tests
tool_results = await test_individual_tools()


🔧 TEST 2: Individual Tool Testing
--------------------------------------------------
🔍 Testing: lookup_customer
   Looking up customer by email
   Args: {'email': 'john.doe@example.com'}
   ✅ Result: User not found....

🔍 Testing: get_available_experiences
   Getting available experiences
   Args: {}
   ✅ Result: No experiences found....

🔍 Testing: search_knowledge_base
   Searching knowledge base
   Args: {'query': 'subscription policy'}
   ✅ Result: Search Error: The api_key client option must be set either by passing api_key to the client or by se...



In [33]:
def create_langchain_tool(mcp_tool, session):
    """Convert MCP tool to LangChain StructuredTool"""

    async def _wrapper(**kwargs):
        result = await session.call_tool(mcp_tool.name, arguments=kwargs)
        return "\n".join([c.text for c in result.content if c.type == "text"])

    return StructuredTool.from_function(
        func=None,
        coroutine=_wrapper,
        name=mcp_tool.name,
        description=mcp_tool.description
    )

print("✅ Helper functions defined")

✅ Helper functions defined


In [34]:
# Test 3: LangChain Tools Integration
print("\n🌐 TEST 3: LangChain Tools Integration")
print("-" * 50)

async def test_langchain_tools():
    """Test LangChain tool wrappers"""

    # Required stderr configuration for Jupyter
    errlog = sys.stderr
    try:
        sys.stderr.fileno()
    except (io.UnsupportedOperation, AttributeError):
        errlog = open(os.devnull, 'w')

    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"],
        env=None
    )

    async with stdio_client(server_params, errlog=errlog) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Get tools and create LangChain wrappers
            tools_result = await session.list_tools()
            langchain_tools = []

            for tool in tools_result.tools:
                lc_tool = create_langchain_tool(tool, session)
                langchain_tools.append(lc_tool)

            print(f"✅ Created {len(langchain_tools)} LangChain tools")

            # Test a LangChain tool call
            search_tool = next((t for t in langchain_tools if t.name == "search_knowledge_base"), None)
            if search_tool:
                print(f"\n🧪 Testing LangChain tool call...")
                try:
                    result = await search_tool.ainvoke({"query": "membership benefits"})
                    print(f"✅ LangChain tool result: {result[:150]}...")
                except Exception as e:
                    print(f"❌ LangChain tool error: {str(e)}")

            return langchain_tools

# Run LangChain tools test
lc_tools = await test_langchain_tools()


🌐 TEST 3: LangChain Tools Integration
--------------------------------------------------
✅ Created 7 LangChain tools

🧪 Testing LangChain tool call...
✅ LangChain tool result: 1 validation error for call[search_knowledge_base]
query
  Missing required argument [type=missing_argument, input_value={}, input_type=dict]
    For ...


In [43]:
from agentic.agents.agents import AgentState

# TEST 4A: Single Agent Testing

In [52]:
from agentic.agents.agent_test_framework import AgentTestFramework, quick_agent_test
from agentic.agents.agents import ToolAgent

In [58]:
async def get_mcp_tools():
    """Get MCP tools directly without manual conversion"""
    server_params = StdioServerParameters(
        command="python",
        args=["mcp_server.py"],
        env=None
    )

    errlog = sys.stderr
    try:
        sys.stderr.fileno()
    except (io.UnsupportedOperation, AttributeError):
        errlog = open(os.devnull, 'w')

    async with stdio_client(server_params, errlog = errlog) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # This automatically converts MCP tools to LangChain tools
            tools = await load_mcp_tools(session)
            return tools

In [59]:
SUBSCRIPTION_SCENARIOS = [
    {
        "query": "I want to check my subscription status. My email is alice.kingsley@example.com",
        "description": "Check subscription status with email lookup",
        "expected_tools": ["lookup_customer", "get_user_subscription"]
    },
    {
        "query": "Can you help me cancel my subscription? My email is john.doe@example.com",
        "description": "Cancel subscription request with email",
        "expected_tools": ["lookup_customer", "cancel_subscription_action"]
    },
    {
        "query": "What's my current subscription tier?",
        "description": "Subscription inquiry without email (should ask for email)",
        "expected_tools": []
    },
    {
        "query": "I need help with billing issues",
        "description": "General subscription support inquiry",
        "expected_tools": ["lookup_customer"]
    }
]

In [60]:
async def test_subscription_agent():
    """Test the Subscription Agent with subscription-related tools"""
    print("\n🧪 TESTING SUBSCRIPTION AGENT")
    print("=" * 50)

    # Get all tools
    all_tools = await get_mcp_tools()

    # Filter tools for subscription agent
    sub_tool_names = ["lookup_customer", "get_user_subscription", "cancel_subscription_action"]
    sub_tools = [t for t in all_tools if t.name in sub_tool_names]

    # Agent configuration
    subscription_config = {
        "name": "Subscription",
        "system_prompt": "You are the Subscription Manager. Always verify user identity by email first using `lookup_customer`. Do not guess IDs. Be helpful and professional."
    }

    # Create test framework
    framework = AgentTestFramework(verbose=True)

    # Run tests
    results = await framework.test_agent(
        agent_class=ToolAgent,
        agent_config=subscription_config,
        tools=sub_tools,
        test_scenarios=SUBSCRIPTION_SCENARIOS
    )

In [61]:
nest_asyncio.apply()

await test_subscription_agent()


🧪 TESTING SUBSCRIPTION AGENT

🧪 Testing Subscription Agent
------------------------------------------------------------
📋 Tools provided: 3
   - lookup_customer
   - get_user_subscription
   - cancel_subscription_action
🎯 Test scenarios: 4
✅ Agent initialized successfully

🔍 Scenario 1: Check subscription status with email lookup
   Query: I want to check my subscription status. My email is alice.kingsley@example.com
   ❌ Error: StructuredTool does not support sync invocation.

🔍 Scenario 2: Cancel subscription request with email
   Query: Can you help me cancel my subscription? My email is john.doe@example.com
   ❌ Error: StructuredTool does not support sync invocation.

🔍 Scenario 3: Subscription inquiry without email (should ask for email)
   Query: What's my current subscription tier?
   ✅ Response: Please provide your email address so I can verify your identity and check your subscription tier.

🔍 Scenario 4: General subscription support inquiry
   Query: I need help with billing

In [15]:
import nest_asyncio

nest_asyncio.apply()

import os
import sys
import io

from mcp import ClientSession, StdioServerParameters

from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools

from agentic.tools.mcp_config import MCP_SERVERS
from agentic.tools.mcp_server import mcp

nest_asyncio.apply()

# UPDATE: Import from the adapter package
server_params = StdioServerParameters(
    command=sys.executable,
    args=["mcp_server.py"],
)

# Ensure we can import from local modules
#sys.path.append(os.getcwd())


async def test_mcp_connection():
    print("🚀 Starting MCP Client Test (via langchain-mcp-adapters)...")

    errlog = sys.stderr
    try:
        sys.stderr.fileno()
    except (io.UnsupportedOperation, AttributeError):
        errlog = open(os.devnull, 'w')

    async with stdio_client(server_params, errlog=errlog) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the connection
            await session.initialize()

            # Get tools
            tools = await load_mcp_tools(session)

            try:

                print("\n🔌 Connecting to server...")
                print("\n--- Testing Subscription Service ---")
                for t in tools:
                    print(f"   - {t.name}: {t.description[:50]}...")

                assert any(t.name == "lookup_customer" for t in tools), "❌ Missing 'lookup_customer'!"

                print("\n--- Testing Reservation Service ---")
                assert any(t.name == "get_available_experiences" for t in tools), "❌ Missing 'get_available_experiences'!"

                print("\n--- Testing Knowledge Service ---")
                assert any(t.name == "search_knowledge_base" for t in tools), "❌ Missing 'search_knowledge_base'!"

            except Exception as e:
                print(f"\n❌ Test Failed: {e}")
                import traceback
                traceback.print_exc()
            finally:
                print("\n🔌 Closing connections...")
                #await client.__aexit__(None, None, None)
                print("✅ Done.")





In [16]:
nest_asyncio.apply()
await test_mcp_connection()

🚀 Starting MCP Client Test (via langchain-mcp-adapters)...

🔌 Connecting to server...

--- Testing Subscription Service ---
   - lookup_customer: Retrieves customer profile by email (ID, Name, Blo...
   - get_user_subscription: Fetches subscription details (Tier, Status, Quota)...
   - cancel_subscription_action: Cancels a user's subscription immediately....
   - get_available_experiences: Lists upcoming experiences that have available slo...
   - get_user_reservations: Lists existing reservations for a user....
   - create_reservation_action: Books an experience for a user if slots are availa...
   - search_knowledge_base: Semantic search for knowledge base articles....

--- Testing Reservation Service ---

--- Testing Knowledge Service ---

🔌 Closing connections...
✅ Done.


In [7]:
import asyncio
loop = asyncio.get_event_loop()
asyncio.run_coroutine_threadsafe(test_mcp_connection(), loop)


<Future at 0x20123cda5d0 state=pending>

🚀 Starting MCP Client Test (via langchain-mcp-adapters)...


# Testing Single Agents

### Testing RAG aka Knowledge base Agent

In [9]:
print("--- Testing RAG Agent ---")
rag_agent = RAGAgent(vector_store).get_graph()

# Test Query
response = rag_agent.invoke({
    "messages": [HumanMessage(content="What is the cancellation policy for premium events?")]
})

print(f"User Question: {response['messages'][0].content}")
print(f"Agent Answer: {response['messages'][-1].content}")

--- Testing RAG Agent ---
User Question: What is the cancellation policy for premium events?
Agent Answer: The cancellation policy for premium events is as follows: 
- 100% refund if cancelled more than 48 hours before the start.
- 50% refund if cancelled between 24 to 48 hours before the start.
- No refund if cancelled less than 24 hours before the start.


# Testing Subscription Agent

In [10]:
print("\n--- Testing Subscription Agent ---")
sub_agent = SubscriptionAgent().get_graph()

# Note: We simulate a user providing their email for context
# Ensure this email exists in your seeded DB (e.g., from 01_external_db_setup)
test_email = "alice.kingsley@wonderland.com"

response = sub_agent.invoke({
    "messages": [
        HumanMessage(content=f"My email is {test_email}. What is my current subscription status?")
    ]
})

print("Conversation History:")
for msg in response['messages']:
    if msg.type == "human":
        print(f"User: {msg.content}")
    elif msg.type == "ai":
        if msg.tool_calls:
            print(f"Agent (Tool Call): {msg.tool_calls[0]['name']}")
        else:
            print(f"Agent: {msg.content}")
    elif msg.type == "tool":
        print(f"Tool Output: {msg.content}")


--- Testing Subscription Agent ---


TypeError: 'FunctionTool' object is not callable

In [11]:
from agentic.tools.mcp_server import (
    lookup_customer,
    get_user_subscription,
    cancel_subscription_action,
    get_available_experiences,
    get_user_reservations,
    create_reservation_action
)

In [14]:
lookup_customer.execution("bob.stone@granite.com")

TypeError: 'NoneType' object is not callable

## Run

In [ ]:
# TODO: Develop your agents under `agentic/agents`
# TODO: Develop your tools under `agentic/tools`
# TODO: Modify `agentic/workflow` in order to orchestrate your agents

In [ ]:
# IDEALLY YOUR ONLY IMPORT HERE IS:
# from agentic.workflow import orchestrator

from agentic.workflow import orchestrator

In [4]:
chat_interface(orchestrator, "1")

User: Hi
Assistant: Hello! How can I assist you today?
User: q
Assistant: Goodbye!


In [5]:
list(orchestrator.get_state_history(
    config = {
        "configurable": {
            "thread_id": "1",
        }
    }
))[0].values["messages"]

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='43c56100-7a4e-4ff0-adb7-1fbbfdac82e3'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 60, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1loRWd5jRqktu5Fut6YZAZsTdc6S', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--0ab48c4a-6130-4dc9-96cd-271e2be7b7c8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 10, 'total_tokens': 70, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]